In [ ]:
# Objective : Given a review, we have to determine the review is either positive(4 or 5) or 
            # Negative (1 or 3)
            # Review (3) - we have to consider as a neutral 
# How to determine if a review is positive or negative ?

In [ ]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
import warnings
warnings.filterwarnings('ignore')

import sqlite3
import nltk
import string

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_curve, auc

from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

# Reading data

In [ ]:
con = sqlite3.connect('database.sqlite')
con

In [ ]:
#filtered_data = pd.read_sql_query(""" select * from Reviews where Score !=3""", con)
#filtered_data1 = pd.read_sql_query(""" select * from Reviews """, con)

In [ ]:
filtered_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [ ]:
filtered_data['Score'].value_counts()

5    363122
4     80655
1     52268
2     29769
Name: Score, dtype: int64

In [ ]:
filtered_data.shape

(525814, 10)

In [ ]:
filtered_data1.shape

(568454, 10)

In [ ]:
filtered_data = pd.read_sql_query("""select * from Reviews where Score !=3 LIMIT 5000""", con)

In [ ]:
filtered_data.shape

(5000, 10)

In [ ]:
def partition(x):
    if x <3:
        return 0
    return 1

actualScore = filtered_data['Score']
PositiveNegative = actualScore.map(partition)
filtered_data['Score'] = PositiveNegative
print("Number of data points in our data", filtered_data.shape)
filtered_data.head(10)

Number of data points in our data (5000, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,1,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,0,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,1,1350777600,Great taffy,Great taffy at a great price. There was a wid...
5,6,B006K2ZZ7K,ADT0SRK1MGOEU,Twoapennything,0,0,1,1342051200,Nice Taffy,I got a wild hair for taffy and ordered this f...
6,7,B006K2ZZ7K,A1SP2KVKFXXRU1,David C. Sullivan,0,0,1,1340150400,Great! Just as good as the expensive brands!,This saltwater taffy had great flavors and was...
7,8,B006K2ZZ7K,A3JRGQVEQN31IQ,Pamela G. Williams,0,0,1,1336003200,"Wonderful, tasty taffy",This taffy is so good. It is very soft and ch...
8,9,B000E7L2R4,A1MZYO9TZK0BBI,R. James,1,1,1,1322006400,Yay Barley,Right now I'm mostly just sprouting this so my...
9,10,B00171APVA,A21BT40VZCCYT4,Carol A. Reed,0,0,1,1351209600,Healthy Dog Food,This is a very healthy dog food. Good for thei...


In [ ]:
filtered_data.head(1)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...


# ***********************************************

In [ ]:
display = pd.read_sql_query("""
select UserId, ProductId, Time, Score, Text, count(*) from reviews
group by UserId Having count(*) >1""", con)

In [ ]:
display.shape

(80668, 6)

In [ ]:
display.tail(10)

,UserId,ProductId,Time,Score,Text,count(*)
80658,AZZH3GGYQSBUC,B000DZKKK2,1229299200,5,"This is good stuff, of which I have consumed a...",2
80659,AZZHZZMH3U1VB,B007TJGZ5E,1350000000,5,This particular brand and blend of Green Mount...,4
80660,AZZJDUEFXYXBM,B001O2HBIM,1284163200,4,My 6.5 month son enjoyed this flavor and it he...,4
80661,AZZMDW27MUJR6,B001FA1L3Y,1256860800,5,It doesn't happen often that I say this but th...,2
80662,AZZMO52V8WZ68,B001E5E20O,1347926400,5,i love this dark acidic tea. a little sugar or...,2
80663,AZZNK89PXD006,B001J3ZUQ6,1269648000,5,In this package you get three varieties of pop...,6
80664,AZZTH6DJ0KSIP,B00014IVPQ,1304208000,5,ive bought a few different kinds and i feel th...,3
80665,AZZU1VEO8KUXH,B003FDC2I2,1317513600,5,I bought this for my 13 year old daughter who ...,3
80666,AZZU4D6TZ2L6J,B0029XITW2,1247875200,5,My father thought this was the best cheese eve...,2
80667,AZZV9PDNMCOZW,B001E5E2QI,1310428800,5,I love this syrup. It is my personal substitut...,3


In [ ]:
display['count(*)'].sum()

393063

In [ ]:
display = pd.read_sql_query("""
select * from reviews where Score !=3 and UserId="AZZTH6DJ0KSIP"
order by productId""", con)

In [ ]:
display

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,128199,B00014IVPQ,AZZTH6DJ0KSIP,Crystal Caccamo,1,1,5,1304208000,coconut oil is the best,ive bought a few different kinds and i feel th...
1,143032,B001AG6BMO,AZZTH6DJ0KSIP,Crystal Caccamo,1,1,5,1304208000,coconut oil is the best,ive bought a few different kinds and i feel th...
2,216633,B003Z4F56C,AZZTH6DJ0KSIP,Crystal Caccamo,7,8,5,1337558400,so awesome~!!!!!,i eat paleo so noodles are not allowed. i trie...


# ***************************************

In [ ]:
filtered_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,1,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,0,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,1,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [ ]:
# Sorting data in ascending order
sorted_data = filtered_data.sort_values('ProductId', ascending=True)

In [ ]:
sorted_data.head(1)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
2546,2774,B00002NCJC,A196AJHU9EASJN,Alex Chaffee,0,0,1,1282953600,thirty bucks?,Why is this $[...] when the same product is av...


In [ ]:
final = sorted_data.drop_duplicates(subset={'UserId', 'ProfileName','Time','Text'}, keep='first')

In [ ]:
final.shape

(4986, 10)

In [ ]:
final['Score'].value_counts()

1    4178
0     808
Name: Score, dtype: int64

In [ ]:
final['Text'][0]

'I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.'

# Text Preprocessing

In [ ]:
sent_0 = final['Text'].values[0]
print(sent_0)
print("="*50)

sent_100 = final['Text'].values[100]
print(sent_100)
print("="*50)

sent_1500 = final['Text'].values[1500]
print(sent_1500)
print("="*50)

sent_2500 = final['Text'].values[2500]
print(sent_2500)
print("="*50)

sent_4500 = final['Text'].values[4500]
print(sent_4500)


Why is this $[...] when the same product is available for $[...] here?<br />http://www.amazon.com/VICTOR-FLY-MAGNET-BAIT-REFILL/dp/B00004RBDY<br /><br />The Victor M380 and M502 traps are unreal, of course -- total fly genocide. Pretty stinky, but only right nearby.
Although this seems like a great product it is definitely not minced.  The cans say it is minced but it isn't, it is ground beef.  My cat ONLY likes minced.  Not chunks, not ground.  Although she will eat ground cat food sometimes, it has to be inviting and delicious, yet she just walks away from this.<br /><br />I hate it when the packaging lies about the contents of a product.  The shipping was speedy and the value is great (as long as they deliver what the promise) now I am stuck with food my cat wont eat at all.
Wow.  So far, two two-star reviews.  One obviously had no idea what they were ordering; the other wants crispy cookies.  Hey, I'm sorry; but these reviews do nobody any good beyond reminding us to look  before o

In [ ]:
import re
sent_0 = re.sub(r"http\S+","",sent_0)
sent_100 = re.sub(r"http\S+","",sent_100)
sent_2500 = re.sub(r"http\S+","",sent_2500)
sent_4500 = re.sub(r"http\S+","",sent_4500)

In [ ]:
print(sent_0)

Why is this $[...] when the same product is available for $[...] here?<br /> /><br />The Victor M380 and M502 traps are unreal, of course -- total fly genocide. Pretty stinky, but only right nearby.


In [ ]:
from bs4 import BeautifulSoup

In [ ]:
soup = BeautifulSoup(sent_100, 'lxml')
text = soup.get_text()
print(text)

Although this seems like a great product it is definitely not minced.  The cans say it is minced but it isn't, it is ground beef.  My cat ONLY likes minced.  Not chunks, not ground.  Although she will eat ground cat food sometimes, it has to be inviting and delicious, yet she just walks away from this.I hate it when the packaging lies about the contents of a product.  The shipping was speedy and the value is great (as long as they deliver what the promise) now I am stuck with food my cat wont eat at all.


In [ ]:
def decontracted(phrase):
    phrase = re.sub(r"isn't", "is not", phrase)
    phrase = re.sub(r"can\'t", "is not", phrase)
    phrase = re.sub(r"\'ve", "is not", phrase)
    phrase = re.sub(r"\'m", "is not", phrase)
    return phrase
    

In [ ]:
sent_100 = decontracted(text)
print(sent_100)

Although this seems like a great product it is definitely not minced.  The cans say it is minced but it is not, it is ground beef.  My cat ONLY likes minced.  Not chunks, not ground.  Although she will eat ground cat food sometimes, it has to be inviting and delicious, yet she just walks away from this.I hate it when the packaging lies about the contents of a product.  The shipping was speedy and the value is great (as long as they deliver what the promise) now I am stuck with food my cat wont eat at all.


In [ ]:

sent_1500 = final['Text'].values[1500]
print(sent_1500)
print("="*50)

Wow.  So far, two two-star reviews.  One obviously had no idea what they were ordering; the other wants crispy cookies.  Hey, I'm sorry; but these reviews do nobody any good beyond reminding us to look  before ordering.<br /><br />These are chocolate-oatmeal cookies.  If you don't like that combination, don't order this type of cookie.  I find the combo quite nice, really.  The oatmeal sort of "calms" the rich chocolate flavor and gives the cookie sort of a coconut-type consistency.  Now let's also remember that tastes differ; so, I've given my opinion.<br /><br />Then, these are soft, chewy cookies -- as advertised.  They are not "crispy" cookies, or the blurb would say "crispy," rather than "chewy."  I happen to like raw cookie dough; however, I don't see where these taste like raw cookie dough.  Both are soft, however, so is this the confusion?  And, yes, they stick together.  Soft cookies tend to do that.  They aren't individually wrapped, which would add to the cost.  Oh yeah, cho

In [ ]:
sent_1500 = re.sub('[^a-zA-Z]+', ' ', sent_1500)
sent_1500 = sent_1500.lower()
print(sent_1500)

wow so far two two star reviews one obviously had no idea what they were ordering the other wants crispy cookies hey i m sorry but these reviews do nobody any good beyond reminding us to look before ordering br br these are chocolate oatmeal cookies if you don t like that combination don t order this type of cookie i find the combo quite nice really the oatmeal sort of calms the rich chocolate flavor and gives the cookie sort of a coconut type consistency now let s also remember that tastes differ so i ve given my opinion br br then these are soft chewy cookies as advertised they are not crispy cookies or the blurb would say crispy rather than chewy i happen to like raw cookie dough however i don t see where these taste like raw cookie dough both are soft however so is this the confusion and yes they stick together soft cookies tend to do that they aren t individually wrapped which would add to the cost oh yeah chocolate chip cookies tend to be somewhat sweet br br so if you want somet

In [ ]:
stopwords = set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [ ]:
# combining all the text 
from tqdm import tqdm

preprocessed_review = []

for sentence in tqdm(final['Text'].values):
    sentence = re.sub(r"http\S+","",sentence)
    sentence = BeautifulSoup(sentence, 'lxml').get_text()
    sentence = decontracted(sentence)
    sentence = re.sub('[^a-zA-Z]+', ' ', sentence)
    sentence = ' '.join(e.lower() for e in sentence.split() if e.lower() not in stopwords)
    preprocessed_review.append(sentence.strip())

100%|████████████████████████████████████████████████████████████████████████████| 4986/4986 [00:04<00:00, 1180.98it/s]


In [ ]:
preprocessed_review[0]

'product available victor traps unreal course total fly genocide pretty stinky right nearby'

In [ ]:
preprocessed_review[1500]

'wow far two two star reviews one obviously no idea ordering wants crispy cookies hey iis not sorry reviews nobody good beyond reminding us look ordering chocolate oatmeal cookies like combination order type cookie find combo quite nice really oatmeal sort calms rich chocolate flavor gives cookie sort coconut type consistency let also remember tastes differ iis not given opinion soft chewy cookies advertised not crispy cookies blurb would say crispy rather chewy happen like raw cookie dough however see taste like raw cookie dough soft however confusion yes stick together soft cookies tend individually wrapped would add cost oh yeah chocolate chip cookies tend somewhat sweet want something hard crisp suggest nabiso ginger snaps want cookie soft chewy tastes like combination chocolate oatmeal give try iis not place second order'

# TF-IDF 

In [ ]:
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=10)
tfidf.fit(preprocessed_review)
print("Some feature name ", tfidf.get_feature_names()[0:10])
print("="*50)
final_counts_tfidf = tfidf.transform(preprocessed_review)
print("The type of count vectorizer", type(final_counts_tfidf))
print("The shape of the text vectorizer", final_counts_tfidf.get_shape())
print("The number of unique words", final_counts_tfidf.get_shape()[1])

Some feature name  ['ability', 'able', 'able find', 'able get', 'absolute', 'absolutely', 'absolutely delicious', 'absolutely love', 'absolutely no', 'according']
The type of count vectorizer <class 'scipy.sparse._csr.csr_matrix'>
The shape of the text vectorizer (4986, 3100)
The number of unique words 3100


In [ ]:
final_counts_tfidf = tfidf.transform(preprocessed_review).toarray()

In [ ]:
final_counts_tfidf.shape

(4986, 3100)

In [ ]:
final_counts_tfidf

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
final.head(1)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
2546,2774,B00002NCJC,A196AJHU9EASJN,Alex Chaffee,0,0,1,1282953600,thirty bucks?,Why is this $[...] when the same product is av...


In [ ]:
final['Score'].shape

(4986,)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(final_counts_tfidf, final['Score'],
                                                   test_size=0.2, random_state=1)

# RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(x_train, y_train)

RandomForestClassifier()

In [ ]:
y_pred_train = rf.predict(x_train)
y_pred_test = rf.predict(x_test)

In [ ]:
print(accuracy_score(y_train, y_pred_train))
print("="*50)
print(accuracy_score(y_test, y_pred_test))

1.0
0.8587174348697395


In [ ]:
print(classification_report(y_train, y_pred_train))
print("="*50)
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       642
           1       1.00      1.00      1.00      3346

    accuracy                           1.00      3988
   macro avg       1.00      1.00      1.00      3988
weighted avg       1.00      1.00      1.00      3988

              precision    recall  f1-score   support

           0       0.79      0.20      0.33       166
           1       0.86      0.99      0.92       832

    accuracy                           0.86       998
   macro avg       0.83      0.60      0.62       998
weighted avg       0.85      0.86      0.82       998

